# Moteur de recherche d’images par le contenu

In [1]:
''' Importation de librairies '''
import cv2 as cv
import numpy as np
import os
from pathlib import Path
import fnmatch
import math
from itertools import islice

In [2]:
''' Chemin vers la base de données d'image '''
base_dir = 'C:\\Users\\Admin\Documents\\eclipse-workspace\\TP_INDEXATION\\Base_test\\'

In [3]:
''' Vérifier si l' image de la requete existe dans la base d'image
    Elle prend on paramètre le nom de l'image avec son extension 
    et retourne vrai ou faux '''
def existe_img_base(img):
    my_file = Path(img)
    if my_file.is_file():
        return True
    else:
        return False

In [4]:
''' Permet de calculer l'histogramme d'une image passée 
    en paramètre et de l'échelle de  réduction
    Nous utiliserons une  réduition l'histograme de 32
    Cette fontion retourne une matrice '''
def construction_histogram(img, echelle_reduction):
    k = [echelle_reduction] * 3
    image = cv.imread(img)
    # Convertir de BGR en RGB
    img_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    histo = cv.calcHist([img_rgb], [0, 1, 2], None,k , [0, 256, 0, 256, 0, 256])
    #histo_normalise = cv.normalize(histo, histo).flatten()
    return histo

In [5]:
''' Calcul de l'histogramme de toute la base de données '''
def calcul_histogramme_base(base,image_requete,echelle_reduction):
    dic_histogramme = {}
    for path, _, files in os.walk(base):
        for file in files:
            #La deuxième condition permet d'exclure l'image de la requete dans le cacul de l'histogramme
            if ((fnmatch.fnmatch(file, '*.png')) and (file != image_requete)):
                fullname = os.path.join(path, file)
                histo = construction_histogram(fullname, echelle_reduction)
                dic_histogramme[file]=histo
    return dic_histogramme

In [6]:
''' Chargement de l'histogramme de toute la base de données '''
histogramme_bases = calcul_histogramme_base(base_dir,'obj1__0.png', 32)

In [7]:
''' Calcul de l'histogramme de l'image de la requete '''
def calcul_histogramme_img_requete(img_requete, echelle_reduction):
    if existe_img_base(img_requete):
        histo = construction_histogram(img_requete, echelle_reduction)
        return histo
    else:
        print("Image inexistante")

In [8]:
''' Chargement de l'histogramme de l'image de test '''
histogramme_img_req = calcul_histogramme_img_requete(base_dir+'obj1__0.png',32)

In [9]:
''' Fonction pour calculer les distance entre l' histogrammes de 
   l'image de requete et l'histogramme des images de la base de 
   données 
   Elle prend en paramètres 2 matrices et retourne la distance
   entre l'image de requete et les images de la bases de données '''
def calcul_distance_histogramme(histo_img_req, histo_bases):
    dic_distance = {}
    for i, j in histo_bases.items():
        distance = cv.compareHist(j,histo_img_req, cv.HISTCMP_CHISQR)
        dic_distance[i]=distance
    return dic_distance

In [10]:
''' Chargement de la  distance de l'image requete
   vers chaque image de  la base '''
distantce_histo = calcul_distance_histogramme(histogramme_img_req,histogramme_bases)

In [11]:
''' Fonction pour calculer le moment d'une image
    Elle prend en paramètre une image et retourne une liste de 7 moments Hu calculé pour chaque image'''
def calcul_moment_hu(img):
    im = cv.imread(img)
    # Convertir l'image en Gray
    img_gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
    # Filtrer l'image pour reduire les bruits:  
    img_gray = cv.blur(img_gray, (3,3))
    ''' Binarisation de l'image à l'aide du seuillage
    _,img_bin = cv.threshold(img_gray, 128, 255, cv.THRESH_BINARY) '''  
    moments = cv.moments(img_gray)
    # Calculate Hu Moments
    huMoments = cv.HuMoments(moments)
    # Log scale hu moments
    for i in range(0,7): 
        huMoments[i] = -1 * math.copysign(1.0, huMoments[i]) * math.log10(abs(huMoments[i]))
    return huMoments

In [12]:
''' Fonction pour calculer les moments des images de la base'''
def calcul_moment_hu_base(base,image_requete):
    dic_moment_hu = {}
    for path, _, files in os.walk(base):
        for file in files:
            if ((fnmatch.fnmatch(file, '*.png')) and (file != image_requete)):
                fullname = os.path.join(path, file)
                moment_hu = calcul_moment_hu(fullname)
                dic_moment_hu[file]=moment_hu
    return dic_moment_hu

In [13]:
''' Chargement des moments des images de la base'''
hu_moment_base = calcul_moment_hu_base(base_dir,'obj1__0.png')

In [14]:
''' Fonction pour calculer les moments des images de la base'''
def calcul_moment_hu_img_requete(img_requete):
    if existe_img_base(img_requete):
        moment_hu = calcul_moment_hu(img_requete)
        return moment_hu
    else:
        print("Image inexistante")

In [15]:
''' Chargement du moment de l'image de test'''
hu_moment_img_req = calcul_moment_hu_img_requete(base_dir+'obj1__0.png')

In [16]:
''' Fonction pour calculer les disctances euclidiennes entre les moments 
   calculer à partir de la base et le moment de l'image de test'''
def calcul_distance_euclidienne(a,b):
    dic_distance = {}
    print(len(a))
    for i, j in a.items():
        dist = np.linalg.norm(j - b)
        dic_distance[i]=dist
    return(dic_distance)

In [17]:
''' Chargement des distances euclidiennes '''
distance_ecludienne = calcul_distance_euclidienne(hu_moment_base,hu_moment_img_req)

14


In [18]:
''' Calcul de la similarité à partir des  distances calculées 
   à partir de l'histogramme (couleur) et la distance euclidienne (forme) '''
def calcul_similarite(dis_histo, dis_eclu, w1, w2):
    dic_similarite = {}
    for i, j in dis_histo.items():
        for k, l in dis_eclu.items():
            if (i == k):
                tot = (j*w1) + (l*w2)
                dic_similarite[i]=tot
                dic_similarite_trie = {keys: val for keys, val in sorted(dic_similarite.items(), key=lambda item: item[1])}
    return(dic_similarite_trie)

In [19]:
''' Fonction pour afficher les k images plus proches de l'image de requete '''
def get_k_plus_proche(k, dic_similarite):
    return list(islice(dic_similarite, k))

### Test avec les valeurs de poids w1 = w2 = 0.5

In [20]:
get_k_plus_proche(7, calcul_similarite(distantce_histo, distance_ecludienne,0.5,0.5).items())

[('obj1__5.png', 1636.6214042191805),
 ('obj1__10.png', 2220.612557816963),
 ('obj5__0.png', 4573.979459380964),
 ('obj5__10.png', 4770.314655474458),
 ('obj5__5.png', 6635.400472389092),
 ('obj4__10.png', 9673.550939378409),
 ('obj4__0.png', 9843.753732680701)]

### Test avec les valeurs de poids w1 = 0.7 et w2 = 0.3

In [21]:
get_k_plus_proche(7, calcul_similarite(distantce_histo, distance_ecludienne,0.7,0.3).items())

[('obj1__5.png', 2272.9967356339603),
 ('obj1__10.png', 3090.8107293423386),
 ('obj5__0.png', 6376.141922714154),
 ('obj5__10.png', 6651.110490866718),
 ('obj5__5.png', 9262.19017596555),
 ('obj4__10.png', 13521.836239294627),
 ('obj4__0.png', 13759.943927296166)]

### Test avec les valeurs de poids w1 = 0.3 et w2 = 0.7

In [22]:
get_k_plus_proche(7, calcul_similarite(distantce_histo, distance_ecludienne,0.3,0.7).items())

[('obj1__5.png', 1000.2460728044008),
 ('obj1__10.png', 1350.4143862915869),
 ('obj5__0.png', 2771.816996047774),
 ('obj5__10.png', 2889.518820082198),
 ('obj5__5.png', 4008.6107688126335),
 ('obj4__10.png', 5825.265639462189),
 ('obj4__0.png', 5927.563538065234)]